In [3]:
import pandas as pd 
import twitter 
from bs4 import BeautifulSoup as bs
import requests

In [4]:
api = twitter.Api(consumer_key = 'Ryx62ngOz4LOFLgOgEYIOLhjd',
                    consumer_secret = '0C7KRFwz4SMHidSdunWyCTtyU2ZRNRaGZIgqQ0Qd6U2Z89bKnz',
                    access_token_key = '1115279873092849664-RL6oaGlrXPaI5ui1zo306BIcRpTAj1',
                    access_token_secret = 'mhYP334f9Xl5olPr3jx2elw2SgL6Vct3LIPDAP1Xax0vS')

In [14]:
def get_twitters(username):
    data = api.GetUserTimeline(screen_name=username, include_rts=False, exclude_replies=False, count=200, max_id="")
    content = [c.text for c in data]
    created_at = [c.created_at for c in data]
    favorite_count = [c.favorite_count for c in data]
    retweet_count = [c.retweet_count for c in data]
    tw_id = [c.id for c in data]
    name = [c.user.screen_name.lower() for c in data]
    result = pd.DataFrame({'username': name, 'tw_id':tw_id, 'content':content, 'created_at':created_at, 'favorite_count': favorite_count, 'retweet_count': retweet_count})
    return result

In [15]:
test = get_twitters('marissamayer')

In [16]:
test.head()

,username,tw_id,content,created_at,favorite_count,retweet_count
0,marissamayer,1134151269147324416,Thank you @LabsLumi for the birthday surprise ...,Thu May 30 17:35:08 +0000 2019,249,9
1,marissamayer,1107696389935398912,Very excited for the launch of #StanfordHumanA...,Mon Mar 18 17:32:54 +0000 2019,246,42
2,marissamayer,1057774580033441793,Happy Halloween! https://t.co/f3UKqn4ZvO,Wed Oct 31 23:21:27 +0000 2018,688,123
3,marissamayer,1025191469093515264,Internapalooza! https://t.co/k21PrT5u1f,Fri Aug 03 01:27:48 +0000 2018,201,12
4,marissamayer,1009254784430272519,"Welcome, Natalie! @n_weyerhaeuser We’re hiring...",Wed Jun 20 02:01:06 +0000 2018,161,13


In [6]:
def parse_lone_replies(replies, reply_to):
    try:
        name = [c.find('div', {'data-component-context':'replies'}).get('data-name') for c in replies]
        tw_id = [c.find("li", {"class": "js-stream-item stream-item stream-item"}).get("data-item-id") for c in replies]
        content = [c.find("p",{"class":"TweetTextSize js-tweet-text tweet-text"}).text for c in replies]
        time = [c.find("a", {"class":"tweet-timestamp js-permalink js-nav js-tooltip"}).get('title') for c in replies]
        tmp2 = [c.find("span", {"class":"ProfileTweet-action--reply u-hiddenVisually"}) for c in replies]
        r_reply = [c.span.span.text for c in tmp2]
        tmp2 = [c.find("span", {"class":"ProfileTweet-action--retweet u-hiddenVisually"}) for c in replies]
        r_retweet = [c.span.span.text for c in tmp2]
        tmp2 = [c.find("span", {"class":"ProfileTweet-action--favorite u-hiddenVisually"}) for c in replies]
        r_likes = [c.span.span.text for c in tmp2]
        result = pd.DataFrame({"r_name": name , "tw_id": tw_id, "content": content, "reply_time": time, 
                               "r_reply": r_reply, "r_retweet": r_retweet, 'r_likes': r_likes, "reply_to":[reply_to]*len(name), "father_tw_id": [reply_to]*len(name)})
        return result 
    except:
        return None

In [7]:
def parse_threaded_replies(replies, reply_to):
    try:
        result = []
        for item in replies:
            tmp2 = item.find_all('li', {"class":'js-stream-item stream-item stream-item'})
            name = [c.find('div', {'data-component-context':'replies'}).get('data-name') for c in tmp2]
            tw_id = [c.get("data-item-id") for c in tmp2]
            content = [c.text for c in item.find_all('p', {"class":"TweetTextSize js-tweet-text tweet-text"})]
            time = [c.get("title") for c in item.find_all("a", {"class":"tweet-timestamp js-permalink js-nav js-tooltip"})]
            r_reply = [c.span.span.text for c in item.find_all("span", {"class":"ProfileTweet-action--reply u-hiddenVisually"})]
            r_retweet = [c.span.span.text for c in item.find_all("span", {"class":"ProfileTweet-action--retweet u-hiddenVisually"})]
            r_likes = [c.span.span.text for c in item.find_all("span", {"class":"ProfileTweet-action--favorite u-hiddenVisually"})]
            reply_to_list = [reply_to] + [tw_id[0]] * (len(tw_id)-1)
            tmp_result = pd.DataFrame({"r_name": name , "tw_id": tw_id, "content": content, "reply_time": time, 
                                   "r_reply": r_reply, "r_retweet": r_retweet, 'r_likes': r_likes, "reply_to":reply_to_list, "father_tw_id":[reply_to]*len(name)})
            result.append(tmp_result)
        result = pd.concat(result)
        return result
    except:
        return None

In [8]:
def get_replies_info(username, tw_id):
    try:
        r = requests.get("https://twitter.com/" + username + "/status/" + str(tw_id))
        html_soup = bs(r.content, "html.parser")
        # parse lone replies
        tmp = html_soup.find_all("li", {"class": "ThreadedConversation--loneTweet"})
        replies_data_lone = parse_lone_replies(tmp, str(tw_id))
        # parse threaded replies
        tmp = html_soup.find_all("li", {"class": "ThreadedConversation"})
        replies_data_threaded = parse_threaded_replies(tmp, str(tw_id))
        replies_data = pd.concat([replies_data_lone, replies_data_threaded])
        return replies_data
    except:
        pass

In [26]:
username = 'marissamayer'
tw_id = 1134151269147324416

In [27]:
test = get_replies_info(username, tw_id)

In [28]:
test

,r_name,tw_id,content,reply_time,r_reply,r_retweet,r_likes,reply_to,father_tw_id
0,Doug Wirebaugh,1134158625759535105,Happy birthday MM,11:04 AM - 30 May 2019,0 replies,0 retweets,3 likes,1134151269147324416,1134151269147324416
1,sigitkka,1134154945337278466,Happy birthday Marissa. I wish you always happ...,10:49 AM - 30 May 2019,0 replies,0 retweets,1 like,1134151269147324416,1134151269147324416
2,Lisa Townsley,1134156618327707648,Happy Birthday,10:56 AM - 30 May 2019,0 replies,0 retweets,1 like,1134151269147324416,1134151269147324416
3,(Mariam) Mary S.,1155855011664015360,that wasn't recognized-we must wish the wish o...,7:58 AM - 29 Jul 2019,0 replies,0 retweets,1 like,1134151269147324416,1134151269147324416
4,Rohan,1134159085996318720,"Happy Birthday, have a great successful year a...",11:06 AM - 30 May 2019,0 replies,0 retweets,0 likes,1134151269147324416,1134151269147324416
5,Ashley Howard,1134166294775779328,Happy birthday Marissa!,11:34 AM - 30 May 2019,0 replies,0 retweets,0 likes,1134151269147324416,1134151269147324416
6,adi kwok,1134169749519122432,happy b'day and glad to hear the lumilabs,11:48 AM - 30 May 2019,0 replies,0 retweets,0 likes,1134151269147324416,1134151269147324416
7,ali,1134170164075880449,Happy birthday Marissa.,11:50 AM - 30 May 2019,0 replies,0 retweets,0 likes,1134151269147324416,1134151269147324416
8,San Vml,1134194509988990977,Hartelijk gefeliciteerd !!,1:26 PM - 30 May 2019,0 replies,0 retweets,0 likes,1134151269147324416,1134151269147324416
9,Ankit Malhotra,1134214319124574208,Happy birthday @marissamayer!,2:45 PM - 30 May 2019,0 replies,0 retweets,0 likes,1134151269147324416,1134151269147324416


In [13]:
with open("local.html", "wb") as f:
    f.write(r.content)

In [14]:
get_replies_info(username, tw_id)

,r_name,content,reply_time,r_reply,r_retweet,r_likes,username,tw_id
0,Dr. Ken Morris,"Well Meg, From where I sit it’s obvious what n...",1:01 PM - 26 Dec 2018,0 replies,0 retweets,2 likes,megwhitman,1077995460260810753
1,Michelle Weston,He is making America great again. He is a tire...,2:41 AM - 27 Dec 2018,0 replies,0 retweets,2 likes,megwhitman,1077995460260810753
2,Chris Fraijo,I dont find anything interesting coming from a...,2:31 AM - 27 Dec 2018,0 replies,0 retweets,1 like,megwhitman,1077995460260810753
3,josephine,I vote for you once but not any more. America ...,10:37 PM - 26 Dec 2018,1 reply,0 retweets,0 likes,megwhitman,1077995460260810753
4,Bawang,"@megwhitman Did you raise entitled, spoiled br...",9:27 AM - 13 Jan 2019,0 replies,0 retweets,0 likes,megwhitman,1077995460260810753
5,The beach is on the left,".@MegWhitman, as a successful businesswoman & ...",6:14 PM - 10 Jan 2019,0 replies,0 retweets,0 likes,megwhitman,1077995460260810753
6,Bawang,"Meg, do you consider yourself a bad mother? Bo...",9:26 AM - 13 Jan 2019,0 replies,0 retweets,0 likes,megwhitman,1077995460260810753
7,श्रवण सिंह राजपुरोहित,https://twitter.com/ShravanSRP/status/98290997...,1:23 AM - 24 Feb 2019,0 replies,0 retweets,0 likes,megwhitman,1077995460260810753
8,Balogun Kehinde,Democracy is Freedom,7:06 AM - 2 Mar 2019,0 replies,0 retweets,0 likes,megwhitman,1077995460260810753
9,Francisco de Lima S.,VASCO,11:36 AM - 17 Mar 2019,0 replies,0 retweets,0 likes,megwhitman,1077995460260810753


In [57]:
def run(username):
    user_twittes = get_twitters(username)
    replies_info = []
    for i,j in user_twittes[['username','tw_id']].values:
        replies_info.append(get_replies_info(i,j))
    replies_info = pd.concat(replies_info)
    reply_count = replies_info.groupby('tw_id').size().to_dict()
    user_twittes['replies_count'] = user_twittes['tw_id'].map(reply_count)
    return user_twittes, replies_info

In [58]:
user_twittes, replies_info = run(username)

In [67]:
with open("social_executives_Lee data added female CEOs.csv",'r') as f:
    user_list = f.readlines()
    user_list = [c.strip().replace(",","") for c in user_list]
    user_list = [c for c in user_list if c is not ""]

In [ ]:
all_user_twittes = []
all_replies_info = []

In [64]:
for username in user_list[1:]:
    user_twittes, replies_info = run(username)
    all_user_twittes.append(user_twittes)
    all_replies_info.append(replies_info)
all_user_twittes = pd.concat(all_user_twittes)
all_replies_info = pd.concat(all_replies_info)
# store to txt
all_user_twittes.to_csv("all_user_twittes.txt", sep="\t", index=None)
all_replies_info.to_csv("all_replies_info.txt", sep='\t', index=None)

# store to csv
all_user_twittes.to_csv("all_user_twittes.csv", index=None)
all_replies_info.to_csv("all_replies_info.csv", index=None)

['\ufeffTwitter',
 'megwhitman',
 'marissamayer',
 'diane_irvine',
 'susansalka',
 'cabachelder',
 'mindygrossman',
 'DawnZier',
 'AndreaJung9',
 'lesaisc',
 'SusanMcGalla',
 'eradicatecancer',
 'BarbaraRentler',
 'GinniRometty ',
 'lisasu',
 'AnnMurtlow',
 'mtbarra',
 'gail_goodman',
 'IndraNooyi',
 'geishajwilliams',
 'SYFMKeane',
 'adenatfriedman',
 'Corie_Barry',
 'poppepk',
 'JulieSweet',
 'BethFordLOL',
 'openjonathan',
 'gcolony',
 'briandunn',
 'roblocascio',
 'jmclaughlin173',
 'lnapier',
 'jdjr2009',
 'jwhitehurst',
 'johnriccitiello',
 'mr2matt',
 'benioff',
 'vivek',
 'wallyboston',
 'kaufer',
 'gregmarcus',
 'davidliuxo',
 'johnheyman',
 'iridiumboss',
 'rsdearth',
 'stevessingh',
 'irvingazoff',
 'mtbert',
 'ericschmidt',
 'jforakerceo',
 'tim_armstrong',
 'learningmoment',
 'michaeldell',
 'aymanashour',
 'hello_joseph',
 'timbiltz',
 'carlbass',
 'michael_saylor',
 'karl_mcdonnell',
 'wymanroberts',
 'davidfieldetm',
 'carlosmcardoso',
 'dkhos',
 'msiegal1',
 'jonaspris